# Initialization

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
from copy import deepcopy
import copy

In [2]:
import argparse
import torch
import numpy as np
import os
import datetime
import torch.nn as nn
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import time
import math

from collections import OrderedDict
from typing import List, Tuple, Union
import matplotlib.pyplot as plt

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

Training on cuda:0


In [3]:
import random
SEED = 42

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

## Set Hyperparameters

In [4]:
input_size = 889
hidden_size = 100
output_size = input_size
batch_size = 32
lr = 0.01
numrounds = 10
USE_CUDA = torch.cuda.is_available()

# Load Data

In [5]:
train_data = np.load(f'./train_data_diginetica.npy', allow_pickle=True)
valid_data = np.load(f'./test_data_diginetica.npy', allow_pickle=True)

In [6]:
train_data[0].columns

Index(['sessionId', 'userId', 'itemId', 'timeframe', 'time', 'userId2',
       'delta_t_a', 'delta_t_b', 'h_a', 'm_a', 's_a', 'h_b', 'm_b', 's_b'],
      dtype='object')

In [7]:
print(f"len train: {len(train_data)}; len test: {len(valid_data)}")

len train: 45; len test: 45


In [8]:
max_user = len(valid_data)
max_user

45

In [9]:
# concat all train data as one dataframe
train_combined = np.concatenate(train_data)
#convert to dataframe
train_combined = pd.DataFrame(train_combined)
train_combined.shape

(1455, 14)

In [10]:
train_combined[2].nunique()

889

In [11]:
# Step 1: Extract unique item IDs from the combined DataFrame
all_unique_items = train_combined[2].unique()

# Step 2: Create a universal item index mapping
universal_item_map = pd.DataFrame({
    'item_idx': np.arange(len(all_unique_items)),
    'itemId': all_unique_items
})

In [12]:
universal_item_map

,item_idx,itemId
0,0,115599.0
1,1,79898.0
2,2,35039.0
3,3,11604.0
4,4,87524.0
...,...,...
884,884,3694.0
885,885,90072.0
886,886,10440.0
887,887,35015.0


# DataLoader Preparation

In [13]:
class Dataset(object):
    def __init__(self, path, sep=',', session_key='sessionId', item_key='itemId', time_key='time', n_sample=-1, itemmap=None, itemstamp=None, time_sort=False):
        # Read csv
        #self.df = pd.read_csv(path, sep=sep, dtype={session_key: int, item_key: int, time_key: float})
        self.df = path
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key
        self.time_sort = time_sort
        if n_sample > 0:
            self.df = self.df[:n_sample]

        # Add colummn item index to data
        self.add_item_indices(itemmap=itemmap)
        """
        Sort the df by time, and then by session ID. That is, df is sorted by session ID and
        clicks within a session are next to each other, where the clicks within a session are time-ordered.
        """
        self.df.sort_values([session_key, time_key], inplace=True)
        self.click_offsets = self.get_click_offset()
        self.session_idx_arr = self.order_session_idx()

    def add_item_indices(self, itemmap=None):
        """
        Add item index column named "item_idx" to the df
        Args:
            itemmap (pd.DataFrame): mapping between the item Ids and indices
        """
        if itemmap is None:
            item_ids = self.df[self.item_key].unique()  # type is numpy.ndarray
            item2idx = pd.Series(data=np.arange(len(item_ids)),
                                 index=item_ids)
            # Build itemmap is a DataFrame that have 2 columns (self.item_key, 'item_idx)
            itemmap = pd.DataFrame({self.item_key: item_ids,
                                   'item_idx': item2idx[item_ids].values})
        self.itemmap = itemmap
        self.df = pd.merge(self.df, self.itemmap, on=self.item_key, how='inner')

    def get_click_offset(self):
        """
        self.df[self.session_key] return a set of session_key
        self.df[self.session_key].nunique() return the size of session_key set (int)
        self.df.groupby(self.session_key).size() return the size of each session_id
        self.df.groupby(self.session_key).size().cumsum() retunn cumulative sum
        """
        offsets = np.zeros(self.df[self.session_key].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby(self.session_key).size().cumsum()
        return offsets

    def order_session_idx(self):
        if self.time_sort:
            sessions_start_time = self.df.groupby(self.session_key)[self.time_key].min().values
            session_idx_arr = np.argsort(sessions_start_time)
        else:
            session_idx_arr = np.arange(self.df[self.session_key].nunique())
        return session_idx_arr
    
    def __len__(self):
        return len(self.session_idx_arr)

    @property
    def items(self):
        return self.itemmap[self.item_key].unique()

In [14]:
class GRUDataset(Dataset):
    def __init__(self, data, itemmap, session_key='sessionId', item_key='itemId', time_key='time'):
        self.data = data
        self.itemmap = itemmap
        self.session_key = session_key
        self.item_key = item_key
        self.time_key = time_key

        # Map items to indices
        self.data = pd.merge(self.data, self.itemmap, on=self.item_key, how='inner')

        # Sort by session and time
        self.data.sort_values([self.session_key, self.time_key], inplace=True)

        # Group data by session and collect item indices
        self.sessions = self.data.groupby(self.session_key)['item_idx'].apply(list)

    def __len__(self):
        return len(self.sessions)

    def __getitem__(self, index):
        session_items = self.sessions.iloc[index]
        sequence = torch.tensor(session_items[:-1], dtype=torch.long)
        target = torch.tensor(session_items[1:], dtype=torch.long)
        return sequence, target

In [15]:
# class DataLoader():
#     def __init__(self, dataset, batch_size=1):
#         """
#         A class for creating session-parallel mini-batches.

#         Args:
#              dataset (SessionDataset): the session dataset to generate the batches from
#              batch_size (int): size of the batch
#         """
#         self.dataset = dataset
#         self.batch_size = batch_size

#     def __iter__(self):
#         """ Returns the iterator for producing session-parallel training mini-batches.

#         Yields:
#             input (B,): torch.FloatTensor. Item indices that will be encoded as one-hot vectors later.
#             target (B,): a Variable that stores the target item indices
#             masks: Numpy array indicating the positions of the sessions to be terminated
#         """
#         # initializations
#         df = self.dataset.df
#         click_offsets = self.dataset.click_offsets
#         session_idx_arr = self.dataset.session_idx_arr

#         iters = np.arange(self.batch_size)
#         maxiter = iters.max()
#         start = click_offsets[session_idx_arr[iters]]
#         end = click_offsets[session_idx_arr[iters] + 1]
#         mask = []  # indicator for the sessions to be terminated
#         finished = False

#         while not finished:
#             minlen = (end - start).min()
#             # Item indices(for embedding) for clicks where the first sessions start
#             idx_target = df.item_idx.values[start]

#             for i in range(minlen - 1):
#                 # Build inputs & targets
#                 idx_input = idx_target
#                 idx_target = df.item_idx.values[start + i + 1]
#                 input = torch.LongTensor(idx_input)
#                 target = torch.LongTensor(idx_target)
#                 yield input, target, mask

#             # click indices where a particular session meets second-to-last element
#             start = start + (minlen - 1)
#             # see if how many sessions should terminate
#             mask = np.arange(len(iters))[(end - start) <= 1]
#             for idx in mask:
#                 maxiter += 1
#                 if maxiter >= len(click_offsets) - 1:
#                     finished = True
#                     break
#                 # update the next starting/ending point
#                 iters[idx] = maxiter
#                 start[idx] = click_offsets[session_idx_arr[maxiter]]
#                 end[idx] = click_offsets[session_idx_arr[maxiter] + 1]

#     def __len__(self):
#         # Return the number of batches in the dataset
#         return (len(self.dataset) + self.batch_size - 1) // self.batch_size

In [16]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def collate_fn(batch):
    sequences, targets = zip(*batch)
    sequences_padded = pad_sequence(sequences, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=-1)
    return sequences_padded, targets_padded

def get_loader(data, itemmap, batch_size=32, shuffle=True):
    dataset = GRUDataset(data, itemmap=itemmap)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=collate_fn)


# Model Architecture

In [17]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        """
        Initialize the LSTM model.

        Args:
            input_size (int): The number of expected features in the input `x`
            hidden_size (int): The number of features in the hidden state `h`
            output_size (int): The size of the output layer (number of items)
            num_layers (int, optional): Number of recurrent layers. Default: 1
        """
        super(LSTMModel, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)

        # LSTM layer
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        """
        Forward pass through the model.

        Args:
            x: Input data
            hidden: Hidden state (h_0, c_0)

        Returns:
            Output and new hidden state
        """
        # Embedding
        embedded = self.embedding(x)

        # LSTM
        output, hidden = self.lstm(embedded, hidden)

        # Predict next item
        output = self.fc(output[:, -1, :])

        return output, hidden

    def init_hidden(self, batch_size):
        """
        Initialize the hidden state of the LSTM.

        Args:
            batch_size (int): The size of the batch

        Returns:
            Initial hidden state (h_0, c_0)
        """
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return (h0, c0)

## Loss Function

In [18]:
class TOP1MaxLoss(torch.nn.Module):
    def __init__(self):
        super(TOP1MaxLoss, self).__init__()

    def forward(self, scores, targets):
        # Initialize loss
        loss = 0.0

        # Loop over each element in the batch
        for i in range(scores.size(0)):  # Loop over batch
            for j in range(targets.size(1)):  # Loop over sequence
                if targets[i, j] == -1:  # Skip padding
                    continue

                # Get the score of the target item
                pos_score = scores[i, targets[i, j]]

                # Calculate the difference with all other items
                diff = -torch.sigmoid(pos_score - scores[i])

                # Exclude the positive item from the loss
                diff[targets[i, j]] = 0

                # Add to the total loss
                loss += torch.sum(diff)

        # Average the loss
        loss = loss / (scores.size(0) * targets.size(1))

        return loss

# Train & Test

In [19]:
def evaluate(net, dataloader, device, k):
    """Evaluate the network on the given data loader for top-k recommendation."""
    net.to(device)
    net.eval()
    criterion = TOP1MaxLoss()  # Replace with your loss function
    total_recall = 0.0
    total_mrr = 0.0
    total_count = 0
    total_loss = 0.0

    with torch.no_grad():
        for x, y in dataloader:
            data, target = x.to(device), y.to(device)
            hidden = net.init_hidden(data.size(0))
            hidden = (hidden[0].to(device), hidden[1].to(device))
            outputs, _ = net(data, hidden)

            # Calculate total loss
            total_loss += criterion(outputs, target)

            # Select top-k items
            _, top_k_indices = torch.topk(outputs, k, dim=1)

            # Calculate recall and MRR for each batch
            for i in range(data.size(0)):
                for target_item in target[i]:
                    if target_item == -1:  # Skip padding or any special token
                        continue
                    target_item_scalar = target_item.item()
                    top_k_items = top_k_indices[i].tolist()

                    # Calculate Recall@k
                    if target_item_scalar in top_k_items:
                        total_recall += 1

                    # Calculate MRR@k
                    if target_item_scalar in top_k_items:
                        rank = top_k_items.index(target_item_scalar)
                        total_mrr += 1 / (rank + 1)

                total_count += len(target[i][target[i] != -1])  # Count non-padding elements

    avg_recall = total_recall / total_count
    avg_mrr = total_mrr / total_count
    avg_loss = total_loss / len(dataloader)

    results = {
        'recall': avg_recall,
        'mrr': avg_mrr
    }

    return avg_loss, results


In [20]:
def train(net, trainloader, epochs, device, valloader=None):
    """Train the network for session-based recommendation."""
    # Define loss and optimizer
    criterion = TOP1MaxLoss()  # Replace with your loss function
    optimizer = torch.optim.Adagrad(net.parameters(), lr= lr)

    print(f"Training {epochs} epoch(s) w/ {len(trainloader)} batches each")
    start_time = time.time()

    net.to(device)
    net.train()

    for epoch in range(epochs):
        total_loss = 0.0

        for x, y in trainloader:
            data, target = x.to(device), y.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            # print(data.shape)
            # print(target.shape)
            # print(target)
            hidden = net.init_hidden(data.size(0))
            hidden = (hidden[0].to(device), hidden[1].to(device))
            outputs, _ = net(data, hidden)

            # Compute loss and backpropagate
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Calculate metrics
        val_loss, val_results = evaluate(net, valloader, device, k=3)

        print(f"Epoch {epoch + 1}: Loss: {total_loss / len(trainloader):.4f}, Recall: {val_results['recall']:.4f}, MRR: {val_results['mrr']:.4f}")

        net.train()  # Ensure the network is in training mode

    total_time = time.time() - start_time
    net.to("cpu")  # Move model back to CPU

    print(f"Training completed in {total_time:.2f} seconds")

    return val_results

# Solo Training

In [21]:
#list of recall and mrr for each user
recall_list = []
mrr_list = []

for i in range(max_user):
    print(f"Training on user {i}...")
    local_train = train_data[i]
    local_test = valid_data[i]

    trainloader = get_loader(local_train, itemmap=universal_item_map, batch_size=batch_size)
    testloader = get_loader(local_test, itemmap=universal_item_map, batch_size=batch_size)

    # Initialize the network
    net = LSTMModel(input_size, hidden_size, output_size, num_layers=3)
    net.to(DEVICE)

    # Train the network
    train_res = train(net, trainloader, numrounds, DEVICE, testloader)

    # Evaluate the network
    loss, results = evaluate(net, testloader, DEVICE, k=5)

    print(f"Recall@5: {results['recall']:.4f}")
    print(f"MRR@5: {results['mrr']:.4f}")

    recall_list.append(results['recall'])
    mrr_list.append(results['mrr'])

print(f"Average Recall@5: {np.mean(recall_list):.4f}")
print(f"Average MRR@5: {np.mean(mrr_list):.4f}")

Training on user 0...


Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: -338.9450, Recall: 0.0000, MRR: 0.0000
Epoch 2: Loss: -394.1923, Recall: 0.5000, MRR: 0.1667
Epoch 3: Loss: -586.8010, Recall: 1.0000, MRR: 0.4167
Epoch 4: Loss: -646.9626, Recall: 0.5000, MRR: 0.2500
Epoch 5: Loss: -653.6553, Recall: 0.5000, MRR: 0.2500
Epoch 6: Loss: -655.7370, Recall: 0.5000, MRR: 0.2500
Epoch 7: Loss: -656.8668, Recall: 0.5000, MRR: 0.2500
Epoch 8: Loss: -657.5597, Recall: 0.5000, MRR: 0.2500
Epoch 9: Loss: -658.0223, Recall: 0.5000, MRR: 0.2500
Epoch 10: Loss: -658.3484, Recall: 1.0000, MRR: 0.4167
Training completed in 0.72 seconds
Recall@5: 1.0000
MRR@5: 0.4167
Training on user 1...
Training 10 epoch(s) w/ 1 batches each
Epoch 1: Loss: -221.2000, Recall: 0.0000, MRR: 0.0000
Epoch 2: Loss: -249.2786, Recall: 0.3333, MRR: 0.1111
Epoch 3: Loss: -384.3234, Recall: 0.3333, MRR: 0.1667
Epoch 4: Loss: -430.4879, Recall: 0.3333, MRR: 0.1667
Epoch 5: Loss: -434.9449, Recall: 0.3333, MRR: 0.1111
Epoch 6: Loss: -436.433

In [22]:
from torchinfo import summary
summary(net)

Layer (type:depth-idx)                   Param #
LSTMModel                                --
├─Embedding: 1-1                         88,900
├─LSTM: 1-2                              242,400
├─Linear: 1-3                            89,789
Total params: 421,089
Trainable params: 421,089
Non-trainable params: 0

In [29]:
print(net)

LSTMModel(
  (embedding): Embedding(889, 100)
  (lstm): LSTM(100, 100, num_layers=3, batch_first=True)
  (fc): Linear(in_features=100, out_features=889, bias=True)
)


# Centralized

In [23]:
#combine all train data as one dataframe
train_combined = np.concatenate(train_data)
train_combined = pd.DataFrame(train_combined)

#set the column name
train_combined.columns = train_data[0].columns


#combine all test data as one dataframe
test_combined = np.concatenate(valid_data)
test_combined = pd.DataFrame(test_combined)

#set the column name
test_combined.columns = valid_data[0].columns

trainloader = get_loader(train_combined, itemmap=universal_item_map, batch_size=batch_size)
testloader = get_loader(test_combined, itemmap=universal_item_map, batch_size=batch_size)

# Initialize the network
net = LSTMModel(input_size, hidden_size, output_size, num_layers=3)
net.to(DEVICE)

# Train the network
train_res = train(net, trainloader, numrounds, DEVICE, testloader)

# Evaluate the network
loss, results = evaluate(net, testloader, DEVICE, k=5)

print(f"Recall@5: {results['recall']:.4f}")
print(f"MRR@5: {results['mrr']:.4f}")


Training 10 epoch(s) w/ 9 batches each
Epoch 1: Loss: -141.9464, Recall: 0.0097, MRR: 0.0032
Epoch 2: Loss: -143.5873, Recall: 0.0000, MRR: 0.0000
Epoch 3: Loss: -188.6809, Recall: 0.0097, MRR: 0.0097
Epoch 4: Loss: -176.9658, Recall: 0.0000, MRR: 0.0000
Epoch 5: Loss: -201.8415, Recall: 0.0000, MRR: 0.0000
Epoch 6: Loss: -196.4691, Recall: 0.0194, MRR: 0.0146
Epoch 7: Loss: -218.6864, Recall: 0.0000, MRR: 0.0000
Epoch 8: Loss: -205.2500, Recall: 0.0194, MRR: 0.0146
Epoch 9: Loss: -209.5740, Recall: 0.0194, MRR: 0.0146
Epoch 10: Loss: -220.2006, Recall: 0.0097, MRR: 0.0097
Training completed in 13.84 seconds
Recall@5: 0.0194
MRR@5: 0.0117


# FL Settings

In [24]:
hidden_size = 100
lr = 0.05
numrounds = 40
num_layers = 2

## Client

In [25]:
class Client():
  def __init__(self, client_config:dict):
    # client config as dict to make configuration dynamic
    self.id = client_config["id"]
    self.config = client_config
    self.__model = None

    # check if CUDA is available
    if torch.cuda.is_available():
      self.device = 'cuda'
    else:
       self.device = 'cpu'

    self.train_loader = self.config["train_data"]
    self.valid_loader = self.config["test_data"]

  @property
  def model(self):
    return self.__model

  @model.setter
  def model(self, model):
    self.__model = model

  def __len__(self):
    """Return a total size of the client's local data."""
    return len(self.train_loader.sampler)

  def train(self):
    results = train(net=self.model,
                    trainloader= self.train_loader,
                    epochs= self.config["local_epoch"],
                    device= self.device,
                    valloader= self.valid_loader)
    print(f"Train result client {self.id}: {results}")

  def test(self):
    loss,result = evaluate(net = self.model,
                    dataloader= self.valid_loader,
                    device=self.device, k=5)
    print(f"Test result client {self.id}: {loss, result}")
    return result

## Server

In [26]:
class FedAvg():
  def __init__(self):
    self.globalmodel = LSTMModel(input_size, hidden_size, output_size, num_layers)
    self.rounds = 0
    self.params = {}

    # check if CUDA is available
    if torch.cuda.is_available():
      self.device = 'cuda'
    else:
       self.device = 'cpu'


  def aggregate(self, round):
    #v1:update the aggregate to save the model with round and date indicator
    modelparams = []
    for i in self.params.keys():
      modelparams.append(self.params[i])

    avg_weights = {}
    for name in modelparams[0].keys():
      avg_weights[name] = torch.mean(torch.stack([w[name] for w in modelparams]), dim = 0)

    self.globalmodel.load_state_dict(avg_weights)

    #current timestamp
    current_time = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    # filename = f"{path_glob_m}/global_model_round_{round}_{current_time}.pth"
    # torch.save(self.globalmodel.state_dict(), filename)

  def clientstrain(self, clientconfig):
    clients = clientconfig
    for i in clients.keys():
      test_client = Client(clients[i])
      test_client.model = copy.deepcopy(self.globalmodel)
      test_client.model.to(self.device)
      test_client.train()
      # test_client.test()
      self.params[i] = test_client.model.state_dict()

  def initiate_FL(self, clientconfig, serverdata):
    clients = clientconfig
    print("Round: {}".format(self.rounds))

    print("Obtaining Weights!!")
    self.clientstrain(clients)

    #### Aggregate model
    print("Aggregating Model!!")
    self.aggregate(self.rounds)

    #### Replace parameters with global model parameters
    for i in self.params.keys():
        self.params[i] = self.globalmodel.state_dict()


    servertest = serverdata
    loss, results = evaluate(net = self.globalmodel,
                    dataloader= servertest,
                    device=self.device, k=5)
    print("Round {} metrics:".format(self.rounds))
    print("Server Loss = {}".format(loss))
    print("Server Recall = {}".format(results['recall']))
    print("Round {} finished!".format(self.rounds))
    self.rounds += 1
    return clients, results['recall']

## Main

In [27]:
clients = {}

for i in range(max_user):
  clients[i] = {"id": i, "val_size": 0.25, "batch_size": batch_size, "local_epoch": 1}
  clients[i]['train_data'] = get_loader(train_data[i], itemmap=universal_item_map, batch_size=batch_size)
  clients[i]['test_data'] = get_loader(valid_data[i], itemmap=universal_item_map, batch_size=batch_size)
  print(f"client: {i}")
  print(f"Number of batches in the dataloader train: {len(clients[i]['train_data'])}")
  print(f"Number of batches in the dataloader test: {len(clients[i]['test_data'])}")

serverdata = get_loader(valid_data[37], itemmap=universal_item_map, batch_size=batch_size)
server = FedAvg() ### initialize server

allrecall = []
for i in range(numrounds):
  clients, recall = server.initiate_FL(clients, serverdata)
  allrecall.append(recall)

print("\n")
print("-" * 50)
print("Recall of all rounds: {}".format(allrecall))

client: 0
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 1
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 2
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 3
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 4
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 5
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 6
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 7
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 8
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 9
Number of batches in the dataloader train: 1
Number of batches in the dataloader test: 1
client: 10

## Test to All Clients

In [28]:
final_model = server.globalmodel
final_model.to(DEVICE)

recall_clients = []
mrr_clients = []
loss_clients = []

# loop for each client
for i in range(max_user):
    print(f"Testing on user {i}...")
    local_test = valid_data[i]
    testloader = get_loader(local_test, itemmap=universal_item_map, batch_size=batch_size)

    # Evaluate the network
    loss, results = evaluate(final_model, testloader, DEVICE, k=5)

    print(f"Recall@5: {results['recall']:.4f}")
    print(f"MRR@5: {results['mrr']:.4f}")

    recall_clients.append(results['recall'])
    mrr_clients.append(results['mrr'])
    loss_clients.append(loss) 

print(f"Average Recall@5: {np.mean(recall_clients):.4f}")
print(f"Average MRR@5: {np.mean(mrr_clients):.4f}")

Testing on user 0...
Recall@5: 0.5000
MRR@5: 0.1667
Testing on user 1...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 2...
Recall@5: 0.5000
MRR@5: 0.1250
Testing on user 3...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 4...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 5...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 6...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 7...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 8...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 9...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 10...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 11...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 12...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 13...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 14...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 15...
Recall@5: 1.0000
MRR@5: 0.4167
Testing on user 16...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 17...
Recall@5: 0.0000
MRR@5: 0.0000
Testing on user 18...
Recall@5: 0.0000
MRR@5: 0.0000
Tes